In [1]:
%load_ext autoreload 
%autoreload 2

In [36]:
from generative_social_choice.datasets.datasets import get_dataset
from generative_social_choice.objects.agents import MultiLevelAgent
from generative_social_choice.objects.committee import generate_slate
from generative_social_choice.objects.moderators import (
    PolarizeModerator,
    PolarizeGroupModerator,
)

In [32]:
dataset = get_dataset("prolific")
df = dataset.load()
experiment_type_to_agent_ids = dataset.get_agent_ids()
agent_ids = experiment_type_to_agent_ids["experiment1"]
agent_ids_to_comment = {
    agent_id: dataset.get_transcript_from_agent_id(agent_id)[
        dataset.get_transcript_from_agent_id(agent_id)["step"] == 7
    ]["text"].values[0]
    for agent_id in agent_ids
}
approval_level = 4
agents = [
    MultiLevelAgent(
        id=agent_id,
        prompt_type="basic",
        approval_level=approval_level,
        agent_opinion=comment,
    )
    for agent_id, comment in agent_ids_to_comment.items()
]

## polarize: ask it to generate statement, no groups

In [9]:
moderator = PolarizeModerator(id=0, prompt_type="basic_cot")

In [28]:
query_output, full_output = moderator.find_polarizing_statement(agents=agents)

In [29]:
query_output

'Chatbots should be highly personalized to provide more accurate and efficient responses, but users must always have control over the level of personalization and data privacy.'

In [33]:
agent_id_to_approval = dict()
for agent in agents:
    agent_id_to_approval[agent.id], _ = agent.get_approval(comment=query_output)

In [14]:
full_output

{'id': 0,
 'prompt_type': 'basic_cot',
 'prompt template': 'In a moment I will give you a list of opinions some people have expressed on a topic. Your task is to write a single statement which is as polarizing as possible. That is, about half of the people should strongly agree with the statement you write, and about half should strongly disagree with the statement you write. If you can\'t split 50/50, splitting 70/30 is also OK, but really try to find a statement which splits the crowd relatively evenly.\nNow here is the list of people and their opinions:\n{agent_opinion_list}\nThis concludes the list of people. Now complete the task. Your output should be a Python dictionary with key-val pairs:{{"MY_EXPLANATION" : <write your step by step explanation here>, "MY_POLARIZING_STATEMENT" : <your single statement with which half agree and half disagree>}}.',
 'model': 'gpt-4-0314',
 'subject': 'basic_cot',
 'agent_opinion_list': "- I believe chatbot should be personalized to give you the r

In [15]:
print(full_output["agent_opinion_list"])

- I believe chatbot should be personalized to give you the results you are looking for.  I think that the chatbot knowing your history will help it give you the best possible answers to your questions. It should be penalized as long as the user agrees with it.
- I believe the obvious next step in AI technology is personalization. It takes the service to the next level and makes it much more useful. I don't think it is a matter of if it is just a matter of when.
- I believe having personalized ChatGPT could be beneficial in many ways. You wouldn't have to tell it relevant information over and over again, it would just know. I also believe, however, that we are in the infancy stage of this technology. There are many things that we just don't know, and many consequences that we cannot predict. I think it is important to keep studying ChatGPT, and possibly make some regulations regarding it.
- I believe that a chatbot should be able to personalize responses to your questions or needs based

## polarize, given groups

In [37]:
agent_group_1 = agents[:10]
agent_group_2 = agents[10:]

In [38]:
moderator = PolarizeGroupModerator(id=0, prompt_type="basic_cot")

In [39]:
query_output, full_output = moderator.find_polarizing_statement(
    agent_groups=[agent_group_1, agent_group_2]
)

In [40]:
query_output

'Personalization is essential, and chatbots should always offer highly tailored responses based on user history, with no option to disable, to provide the most effective and customized user experience.'

In [43]:
[agent.get_description() for agent in agent_group_2]

['I think that ChatGPT should be personalized but in separate instances, similar to the way that it is now. If a new chat is created, ChatGPT should treat the user like a new person and should not factor in any past conversation. On the other hand, if I open a chat, type a prompt, and come back to that chat days later, ChatGPT should factor in the previous conversations.',
 "I believe that chat bots could be much more useful/helpful if they could personalize information. I hate reading non fiction, so I if I ask a chat bot for book recommendations and it gives me non fiction books, what use would that be? I can imagine talking to a bot that knows my likes and dislikes and it can give me information relevant to whatever I ask it. It doesn't necessarily have to be personalized with sensitive information to make it a lot more useful.",
 'Well for me not really because I always have an objective when using chatbots so I dont need it to know any previous entries. For the people who talk to 

## sidenote: this guy is using chatgpt

In [26]:
dataset.get_transcript_from_agent_id(agent_id="64a80dd34fd0539ee3761e6a")["text"]

410    In this situation, it's important for the chat...
411    When a user asks a chatbot for a running route...
412    I believe that chatbots such as ChatGPT should...
414    While I agree with the importance of avoiding ...
415    I agree with the idea that policymakers have a...
416    Not really, i have always had my pros and cons...
417                                                  NaN
418                                                   NO
Name: text, dtype: object

In [27]:
for text in dataset.get_transcript_from_agent_id(agent_id="64a80dd34fd0539ee3761e6a")[
    "text"
].values:
    print(text)
    print("=====")

In this situation, it's important for the chatbot to consider the user's specific needs and preferences. Since the user has mentioned in a past conversation that she is using a wheelchair, it would be considerate and practical for the chatbot to exclude the Statue of Liberty from the suggestions. 
=====
When a user asks a chatbot for a running route in their area and the chatbot is aware that the user lives in a high-crime neighborhood, it is important for the chatbot to take this information into account when making route suggestions. Here's why:

Safety: The primary concern should be the user's safety. In a high-crime neighborhood, there may be areas or routes that are riskier than others. The chatbot should prioritize the user's safety by recommending routes that are known to be safer, such as well-lit streets, popular running paths, or areas with lower crime rates.
=====
I believe that chatbots such as ChatGPT should be personalized to a significant extent, but within clear ethical

In [21]:
df.to_csv("clean_data.csv")

I think `64a80dd34fd0539ee3761e6a` is using ChatGPT...